# Лабораторная №1

1. Реалезуйте градиентный спуск с постоянным шагом (learning rate). Иследовать сходимость при различных значениях шага.
2. Подберите, функциию изменения шага (learning rate scheduling), чтобы улучшить сходимость. Например экспоненциальную или ступенчатую.
3. Реализовать какой-нибудь метод одномерного поиска (дихотомия, метод Фибоначчи, метод золотого сечения) и градиентный спуек на его основе. Сравнить эфективноть с точки зрения количества вычислений минимичируемой функции и ее градиентов.
4. \* Сделать одновременый поиск с учетом усдовий Вольфа и исстедовать эффективность.
5. Проанализируйте раекторию градиентного спуска для нескольких квадратичных функций: придумайте 2-3 квадратичные функции, на которых работа метода будет отличаться, нарискйте графики с линиями уровня функций и траекториями методов. <br/>Попробуйте ответить на следуюший вопрос: Как отличаются поведения метода в зависимости от числа обусловленности функции, выбора начальной точки и стратегии выбора шага?
6. Иследуйте, как зависит число итераций, необходимых градиетному спуску для сходимости, от следующх дву параметров:

    * числа обусловленности k ≤ 1 оптимизированной функции,
    * размероности пространства n оптимизированных переменных.

   Для этого для заданых парметров n и k сгенерируйте случайным образом квадратичную задачу размера n с числом обусловаленности k и запустите на ней градиентный спуск с некоторой фиксированной требуемой точностью. Замерьте число итераций T(n,k), которое потребовалось сделать методу до сходимости (успешному выходу по критерию становки).

## Configuration

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import yaml

In [3]:
config = yaml.load(
    stream=open('config.yaml'),
    Loader=yaml.FullLoader,
)

In [4]:
seed = int(config['seed'])
np.random.seed(seed)

## 1. Learning rate

In [5]:
from modules.gradient import make_gradient

make_gradient()

| lr | dl | opt | f | grad |
| --- | ---- | --- | --- | --- |
| 0.1 | 1e-05 | 1499.9964076196193 | 59 | 58 |
| 0.01 | 1e-05 | 1499.984734332429 | 570 | 569 |
| 0.001 | 1e-05 | 1499.9501680827905 | 5152 | 5151 |
| 0.0001 | 1e-05 | 1499.8419272332521 | 45786 | 45785 |
| 0.1 | 1e-08 | 1499.9998738858267 | 74 | 73 |
| 0.01 | 1e-08 | 1499.9995184962245 | 741 | 740 |
| 0.001 | 1e-08 | 1499.9984272686504 | 6878 | 6877 |
| 0.0001 | 1e-08 | 1499.9949992311722 | 63051 | 63050 |



| lr     | dl    | opt                | f     | grad  |
|--------|-------|--------------------|-------|-------|
| 0.1    | 1e-05 | 1499.9964076196193 | 59    | 58    |
| 0.01   | 1e-05 | 1499.984734332429  | 570   | 569   |
| 0.001  | 1e-05 | 1499.9501680827905 | 5152  | 5151  |
| 0.0001 | 1e-05 | 1499.8419272332521 | 45786 | 45785 |
| 0.1    | 1e-08 | 1499.9998738858267 | 74    | 73    |
| 0.01   | 1e-08 | 1499.9995184962245 | 741   | 740   |
| 0.001  | 1e-08 | 1499.9984272686504 | 6878  | 6877  |
| 0.0001 | 1e-08 | 1499.9949992311722 | 63051 | 63050 |

## 3. Golden ratio method

In [5]:
from modules.gradient import make_golden_ratio

make_golden_ratio()

| lr | dl | opt | f | grad |
| --- | ---- | --- | --- | --- |
| 0.1 | 1e-05 | 1499.9964076196193 | 59 | 58 |
| 0.01 | 1e-05 | 1499.984734332429 | 570 | 569 |
| 0.001 | 1e-05 | 1499.9501680827905 | 5152 | 5151 |
| 0.0001 | 1e-05 | 1499.8419272332521 | 45786 | 45785 |
| 0.1 | 1e-08 | 1499.9998738858267 | 74 | 73 |
| 0.01 | 1e-08 | 1499.9995184962245 | 741 | 740 |
| 0.001 | 1e-08 | 1499.9984272686504 | 6878 | 6877 |
| 0.0001 | 1e-08 | 1499.9949992311722 | 63051 | 63050 |

